The goal of this analysis is to figure out three things:

1) How many guides are affected in at least n cell lines
2) How many cell lines are affected for each guide
3) questions #1 and #2 but for genes

In [1]:
#Set up the environment
library(tidyverse)
library(reshape2)

#Define functions
show_msg <- function(x){ 
    print(x)
    flush.console()
}

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [4]:
#We've already subset the vcf file to only include SNVs that map to Avana guide targeting sequences in a prior analysis (New_SNV_Position_Single_Guide_Association.ipynb).
#So let's just load that output so that we don't need to re-compute things.
setwd("/home/jupyter/Ancestry/analysis/map_ccle_snps_to_avana_guides")
ccle.sample.names = read.table("ccle.sample.names.txt", sep = "\t", header = FALSE) %>% pull(1)
ccle.avana.genotype.matrix = read.table("snps.in.all.avana.guides.vcf.gz", sep = "\t", header = FALSE) %>%
select(-1, -2, -4, -5, -6, -7, -8, -9)
colnames(ccle.avana.genotype.matrix) = c("snp", ccle.sample.names)

head(ccle.avana.genotype.matrix)

,snp,ACH-000657,ACH-000617,ACH-000685,ACH-000632,ACH-000171,ACH-000863,ACH-000971,ACH-000178,ACH-000822,⋯,ACH-000981,ACH-000217,ACH-000116,ACH-000930,ACH-000986,ACH-000342,ACH-001130,ACH-000287,ACH-000798,ACH-000645
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chr1:1407014:G:A,"0|0:0:0,0:1,0,0","0|0:0.009:0.002,0.007:0.991,0.009,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0.001:0,0.001:0.999,0.001,0","0|0:0.038:0.038,0:0.962,0.038,0","0|0:0:0,0:1,0,0","0|0:0.011:0.009,0.002:0.989,0.011,0","0|0:0:0,0:1,0,0",⋯,"0|0:0.005:0,0.005:0.995,0.005,0","0|0:0:0,0:1,0,0","0|0:0.001:0,0.001:0.999,0.001,0","0|0:0:0,0:1,0,0","0|0:0.002:0.002,0:0.998,0.002,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0.007:0,0.007:0.993,0.007,0","0|0:0.003:0.003,0:0.997,0.003,0"
2,chr1:1471992:T:C,"1|1:1.965:0.965,1:0,0.035,0.965","1|1:1.08:0.517,0.564:0.211,0.498,0.291","0|1:0.804:0,0.804:0.196,0.803,0","0|0:0.147:0.141,0.006:0.853,0.146,0.001","0|1:0.667:0,0.666:0.334,0.666,0","0|0:0.46:0.46,0:0.54,0.46,0","0|0:0.039:0.023,0.016:0.962,0.038,0","0|0:0.03:0.026,0.004:0.971,0.029,0","0|1:0.893:0.008,0.885:0.114,0.879,0.007",⋯,"1|0:0.993:0.676,0.318:0.221,0.564,0.215","1|1:1.559:0.944,0.615:0.022,0.398,0.581","1|1:1.771:0.989,0.782:0.002,0.224,0.774","1|1:1.983:0.999,0.984:0,0.017,0.983","0|0:0.019:0.007,0.012:0.981,0.019,0","1|0:1.123:0.644,0.479:0.185,0.506,0.309","0|1:1.255:0.433,0.822:0.101,0.543,0.356","1|0:0.979:0.975,0.003:0.024,0.972,0.003","0|0:0.025:0.004,0.021:0.975,0.025,0","0|0:0.067:0.059,0.009:0.933,0.066,0.001"
3,chr1:1918305:G:A,"1|0:0.965:0.965,0:0.035,0.965,0","0|0:0.005:0.001,0.005:0.995,0.005,0","0|0:0.128:0,0.128:0.872,0.128,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0",⋯,"0|0:0:0,0:1,0,0","0|0:0.003:0,0.003:0.997,0.003,0","0|0:0:0,0:1,0,0","0|0:0.217:0,0.217:0.783,0.217,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0.001:0,0.001:0.999,0.001,0","1|0:0.974:0.974,0:0.026,0.974,0","1|0:1:1,0:0,1,0","0|1:0.998:0,0.998:0.002,0.998,0"
4,chr1:2028260:G:A,"0|0:0:0,0:1,0,0","0|0:0.236:0.182,0.054:0.774,0.217,0.01","0|0:0:0,0:1,0,0","0|0:0.036:0.021,0.015:0.964,0.036,0","0|0:0:0,0:1,0,0","0|0:0.017:0.001,0.016:0.983,0.017,0","0|0:0:0,0:1,0,0","0|0:0.01:0.01,0:0.99,0.01,0","0|0:0.001:0,0.001:0.999,0.001,0",⋯,"0|0:0.005:0,0.005:0.995,0.005,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0.007:0.002,0.005:0.993,0.007,0","0|0:0.027:0.021,0.005:0.974,0.026,0","0|0:0.053:0.034,0.019:0.948,0.051,0.001","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0"
5,chr1:2351722:A:G,"1|1:2:1,1:0,0,1","0|0:0:0,0:1,0,0","0|1:0.991:0,0.991:0.009,0.991,0","0|1:1:0,1:0,1,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0",⋯,"0|0:0:0,0:1,0,0","0|1:0.992:0,0.992:0.008,0.992,0","0|0:0:0,0:1,0,0","1|1:2:1,1:0,0,1","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|0:0:0,0:1,0,0","0|1:1:0,1:0,1,0","0|0:0:0,0:1,0,0"
6,chr1:2408761:T:C,"1|0:1:1,0:0,1,0","1|1:1.834:0.927,0.907:0.007,0.152,0.841","1|1:1.714:1,0.714:0,0.286,0.714","1|1:1.783:0.941,0.842:0.009,0.198,0.793","1|1:1.997:1,0.998:0,0.003,0.997","1|1:1.85:0.91,0.939:0.005,0.14,0.855","0|1:0.973:0,0.973:0.027,0.973,0","1|1:1.884:0.944,0.94:0.003,0.109,0.888","0|1:1.024:0.03,0.995:0.005,0.965,0.029",⋯,"1|1:1.979:0.985,0.994:0,0.02,0.979","1|1:1.8:1,0.8:0,0.2,0.8","1|1:1.958:0.994,0.965:0,0.041,0.958","1|1:1.991:0.999,0.992:0,0.009,0.991","1|0:0.989:0.989,0:0.011,0.989,0","1|0:0.776:0.772,0.003:0.227,0.771,0.002","0|0:0.05:0.025,0.025:0.951,0.049,0.001","1|1:1.932:0.932,1:0,0.068,0.932","0|1:0.996:0,0.996:0.004,0.996,0","1|1:1.999:1,0.999:0,0.001,0.999"


In [5]:
#Process the genotype matrix to re-format how the genotypes are encoded
#Reformat the genotyping data
ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)] = apply(ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)], 2, function(x){gsub(":..*", "", x) })
ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)] = apply(ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)], 2, function(x){gsub("0\\|0", 0, x) })
ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)] = apply(ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)], 2, function(x){gsub("0\\|1", 1, x) })
ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)] = apply(ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)], 2, function(x){gsub("1\\|0", 1, x) })
ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)] = apply(ccle.avana.genotype.matrix[2:ncol(ccle.avana.genotype.matrix)], 2, function(x){gsub("1\\|1", 2, x) })

#Fix the SNP column
ccle.avana.genotype.matrix = ccle.avana.genotype.matrix %>%
separate(snp, sep = ":", into = c("chr", "pos", "ref", "alt"))

head(ccle.avana.genotype.matrix)

,chr,pos,ref,alt,ACH-000657,ACH-000617,ACH-000685,ACH-000632,ACH-000171,ACH-000863,⋯,ACH-000981,ACH-000217,ACH-000116,ACH-000930,ACH-000986,ACH-000342,ACH-001130,ACH-000287,ACH-000798,ACH-000645
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chr1,1407014,G,A,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,chr1,1471992,T,C,2,2,1,0,1,0,⋯,1,2,2,2,0,1,1,1,0,0
3,chr1,1918305,G,A,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,1,1,1
4,chr1,2028260,G,A,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,chr1,2351722,A,G,2,0,1,1,0,0,⋯,0,1,0,2,0,0,0,0,1,0
6,chr1,2408761,T,C,1,2,2,2,2,2,⋯,2,2,2,2,1,1,0,2,1,2


In [6]:
#Load in the avana bed file
setwd("/home/jupyter/Ancestry/filtered_output/avana14")
avana.bed <- read.table('Avana14_filtering.bed', sep = "\t", header = F) %>%
rename("chr" = 1, "start" = 2, "end" = 3, "sgrna" = 4) 
head(avana.bed)

,chr,start,end,sgrna
,<chr>,<int>,<dbl>,<chr>
1,chr1,69527,69550,TCTCCATTCGGTGAGCCAGT
2,chr1,168646,168669,TGTAGTCCCAGCTACTCAGG
3,chr1,383420,383443,GAGAATCTCTTGAACCCGGG
4,chr1,424270,424293,TGTAATCCCAGTACTTTGGG
5,chr1,451271,451294,ATAATGGTCAGATAGTGGAG
6,chr1,451344,451367,TCTCCACACCACCAACGACG


In [7]:
#load in the guide map so that we can figure out which guides are on the plus or minus strand
setwd("/home/jupyter/Ancestry/gwas")
guide.map = read.table("internal-21q4_v9-achilles-guide-map.csv", header = T, sep = ",") %>%
select(sgrna, genome_alignment) %>%
mutate(genome_alignment = gsub("..*_", "", genome_alignment)) %>%
rename("strand" = genome_alignment) %>%
distinct(sgrna, .keep_all = TRUE)
head(guide.map)

#Load in the Avana bed file to get positions for the ancestry-associated guides
#Also add the annotation information from guides.to.keep to the avana.bed matrix
#Also flip the start and end positions if the sgrna is on the minus strand
setwd("/home/jupyter/Ancestry/filtered_output/avana14")
avana.bed <- read.table('Avana14_filtering.bed', sep = "\t", header = F) %>%
rename("chr" = 1, "start" = 2, "end" = 3, "sgrna" = 4) %>%
inner_join(guide.map, by = "sgrna") %>%
mutate("true_start" = ifelse(strand %in% "+", start, end)) %>%
mutate("true_end" = ifelse(strand %in% "+", end, start))%>% #This line and the line above will create two new columns to flip the start and end if the sgrna is on the minus strand
select(chr, true_start, true_end, sgrna) %>%
rename("start" = true_start) %>%
rename("end" = true_end)

tail(avana.bed)
dim(avana.bed)

,sgrna,strand
,<chr>,<chr>
1,AAAAAAATCCAGCAATGCAG,+
2,AAAAAACCCGTAGATAGCCT,+
3,AAAAAAGAAGAAAAAACCAG,-
4,AAAAAAGCTCAAGAAGGAGG,-
5,AAAAAAGGCTGTAAAAGCGT,+
6,AAAAAAGGGCTCCAAAAAGG,+


,chr,start,end,sgrna
,<chr>,<dbl>,<dbl>,<chr>
72099,chrX,155514278,155514256,TTCCCCTTAAAAGGGCCAGA
72100,chrX,155773991,155773969,GGAGAGGGCCTGTTTACAGG
72101,chrX,155774016,155773994,GGGTTTGCACAATAAGGGAA
72102,chrX,155774310,155774288,AGATGAAGAGGTGCTCACTA
72103,chrX,156001472,156001450,AGCCACAGCATGAAACACCG
72104,chrX,156003813,156003791,GCTCCCTCCCAGACATGCAG


[1] 72104     4

In [19]:
#For each guide in avana.bed calculate if there is a snp in the guide, then calculate the position of that SNP
snp.in.guide.list = NULL
for(i in 1:nrow(avana.bed)){
  
    #get information for the guide of interest
    guide.start = avana.bed$start[i] %>% as.numeric() #get the position of the guide
    guide.chrom = avana.bed$chr[i] #get the chromosome that the guide is on
    sgrna.barcode = avana.bed$sgrna[i] #get the barcode sequence
    
    #subset out subset.ccle.snps so that it only includes snps on that chromsome
    subset.to.chr = ccle.avana.genotype.matrix %>% filter(chr %in% guide.chrom)
    
    #now calculate the differential and add it to the list outside of the loop
    snp.position = guide.start - as.numeric(subset.to.chr$pos)
    snp.in.guide.list[[i]] = snp.position
}
names(snp.in.guide.list) = avana.bed$sgrna #Then name the list with the corresponding sgrna sequence

In [20]:
#Assign the SNP ID as names for each vector

annotated.snp.in.guide.list = NULL
for(i in 1:length(snp.in.guide.list)){
        
    chromosome = avana.bed$chr[i] #get the chromosome which that guide targets
    
    #Now get a vector of snp positions and annotate the relative snp position vector with that information
    vector.to.annotate = snp.in.guide.list[[i]] #grab the vector that we need to annotate in this loop
    snp.vector = ccle.avana.genotype.matrix %>% filter(chr %in% chromosome) %>% mutate(snp = paste(chr, ":", pos, ":", ref, ":", alt, sep = "")) %>% pull(snp) #make a vector that contains the snp information
    names(vector.to.annotate) = snp.vector #now apply that annotation to the vector of snp positions
    
    #Report the information outside of the loop
    annotated.snp.in.guide.list[[i]] = vector.to.annotate
    
}

names(annotated.snp.in.guide.list) = avana.bed$sgrna #Then name the list with the corresponding sgrna sequence

In [21]:
#Go through the list and remove the SNPs that don't map to a sgrna
pruned.snp.list = annotated.snp.in.guide.list %>%
lapply(function(x){
    vector = abs(x)
    val.to.return = vector[vector %in% 0:22]
    return(val.to.return)
}) %>% compact(1) #compact is part of the purrr package and will remove guides that don't have a SNP

head(pruned.snp.list)
length(pruned.snp.list)

$GGGATGACTGCCACGTCGCA
chr1:1918305:G:A 
              15 

$CACCAGCTACCGCTTCACCA
chr1:2028260:G:A 
              12 

$TGAACTCATCACAATCACAG
chr1:2351722:A:G 
              15 

$GCAGTGTCACCAGCACGCCA
chr1:2408761:T:C chr1:2408773:G:A 
               5               17 

$GCTCAGGGACCACAGTCCCG
chr1:2645520:C:T 
              18 

$CGTACGTTCCTTCTAGAGCA
chr1:3844915:C:T 
               5

[1] 3742

In [22]:
#Now let's go through and create a nice data frame from this list.

snp.in.guide.df = NULL
for(i in 1:length(pruned.snp.list)){
    
   sgrna = pruned.snp.list[[i]] #this is a vector (normally length = 1, sometimes more with all of the snps that map to a vector)
   
    #If there is only one snp that maps to that sgrna, simply construct the vector and add it to the data frame
    #If there is more than one snp, then we need to get a little more fancy and construct a data frame first
    if(length(sgrna) == 1){
        sgrna = names(pruned.snp.list[i]) #get the sgrna for this iteration of the loop
        snp = names(pruned.snp.list[[i]]) #get the snp code for this iteration of the loop
        position = pruned.snp.list[[i]] #Get the relative position of the SNP on the guide for this iteration of the loop
        vector.to.add = c(sgrna, snp, position) #Make a vector of values to add to the data frame
        snp.in.guide.df = rbind(snp.in.guide.df, vector.to.add) %>% data.frame() %>% #Then add that vector to the data frame
        rename("sgrna" = 1, "snp" = 2, "position" = 3)  
    } else {
        sgrna = names(pruned.snp.list[i]) #get the sgrna for this iteration of the loop
        sgrna.vector = rep(sgrna, length(pruned.snp.list[[i]])) #get a vector that contains the sgrna now, length = #snps
        snp.vector = names(pruned.snp.list[[i]]) #get the snp code for this iteration of the loop
        position.vector = pruned.snp.list[[i]] #Get the relative position of the SNP on the guide for this iteration of the loop
        df.to.add = cbind(sgrna.vector, snp.vector, position.vector) %>% data.frame() %>%
        rename("sgrna" = 1, "snp" = 2, "position" = 3) #bind it all into a data frame
        snp.in.guide.df = rbind(snp.in.guide.df, df.to.add) %>% data.frame() %>%
        rename("sgrna" = 1, "snp" = 2, "position" = 3)
 }
}

row.names(snp.in.guide.df) = NULL #remove the row names on the new df since they are useless
head(snp.in.guide.df)
dim(snp.in.guide.df)

,sgrna,snp,position
,<chr>,<chr>,<chr>
1,GGGATGACTGCCACGTCGCA,chr1:1918305:G:A,15
2,CACCAGCTACCGCTTCACCA,chr1:2028260:G:A,12
3,TGAACTCATCACAATCACAG,chr1:2351722:A:G,15
4,GCAGTGTCACCAGCACGCCA,chr1:2408761:T:C,5
5,GCAGTGTCACCAGCACGCCA,chr1:2408773:G:A,17
6,GCTCAGGGACCACAGTCCCG,chr1:2645520:C:T,18


[1] 3961    3

In [46]:
#Write this dataframe so that we don't need to re-run all of this code
setwd("/home/jupyter/Ancestry/analysis/new_ccle_affected_guides_analysis")
write.table(snp.in.guide.df, "snp.in.guide.df.txt", sep = "\t", col.names = TRUE, row.names = FALSE, quote = FALSE)

In [2]:
#Read it back in
setwd("/home/jupyter/Ancestry/analysis/new_ccle_affected_guides_analysis")
snp.in.guide.df = read.table("snp.in.guide.df.txt", header = T, sep = "\t")
head(snp.in.guide.df)

,sgrna,snp,position
,<chr>,<chr>,<int>
1,GGGATGACTGCCACGTCGCA,chr1:1918305:G:A,15
2,CACCAGCTACCGCTTCACCA,chr1:2028260:G:A,12
3,TGAACTCATCACAATCACAG,chr1:2351722:A:G,15
4,GCAGTGTCACCAGCACGCCA,chr1:2408761:T:C,5
5,GCAGTGTCACCAGCACGCCA,chr1:2408773:G:A,17
6,GCTCAGGGACCACAGTCCCG,chr1:2645520:C:T,18


Make a plot that indicates how many cell lines have a SNP in each guide

In [ ]:
#Filter the matrix so that it only includes cell lines that were profiled with CRISPR screens
#First load in the chronos data
setwd("/home/jupyter/Ancestry/analysis/map_ccle_snps_to_avana_guides")
depmap.lines = read.table("CRISPR_gene_effect_22Q1.csv", sep = ",", header = T) %>%
pull(DepMap_ID)
head(depmap.lines)

In [8]:
#First process the matrix so that it is in the correct format.
processed.genotype.matrix = ccle.avana.genotype.matrix %>%
mutate("snp" = paste(chr, ":", pos, ":", ref, ":", alt, sep = ""), .before = chr) %>%
select(-chr, -pos, -ref, -alt) %>%
mutate(snp = plyr::mapvalues(snp, from = snp.in.guide.df$snp, to = snp.in.guide.df$sgrna, warn_missing = FALSE)) %>%
mutate_at(vars(2:ncol(.)), as.numeric) %>%
aggregate(. ~ snp, FUN = sum)


head(processed.genotype.matrix)
dim(processed.genotype.matrix)



,snp,ACH-000657,ACH-000617,ACH-000685,ACH-000632,ACH-000171,ACH-000863,ACH-000971,ACH-000178,ACH-000822,⋯,ACH-000981,ACH-000217,ACH-000116,ACH-000930,ACH-000986,ACH-000342,ACH-001130,ACH-000287,ACH-000798,ACH-000645
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AAAACTGTACGAGTGACGGA,0,0,0,1,1,1,2,0,0,⋯,2,0,1,1,0,0,1,1,1,2
2,AAAGCGGTCACCTTCCACAT,0,2,0,1,0,0,1,0,0,⋯,0,0,0,0,0,0,1,1,1,1
3,AAAGGAGTCGAAGAAGTCCG,0,2,1,2,1,2,0,1,1,⋯,2,2,2,1,1,1,1,0,1,1
4,AAAGGATGAAAGACTCCGAA,2,2,2,2,2,2,1,0,2,⋯,1,2,1,2,0,2,2,2,2,2
5,AAAGTGCAAGAGGATACCCA,2,2,2,2,2,2,1,0,2,⋯,2,0,2,2,2,2,0,2,2,2
6,AAAGTTTCTGTCTTACCTGG,1,2,1,0,0,2,2,1,2,⋯,0,2,0,2,2,1,1,2,1,1


[1] 3274  995

In [27]:
#Now filter the data frame
lines.to.keep = intersect(depmap.lines, colnames(processed.genotype.matrix))

processed.genotype.matrix = processed.genotype.matrix %>%
select(snp, all_of(lines.to.keep))

dim(processed.genotype.matrix)

[1] 3274  634

In [28]:
#At this point the numbers are meaningless. But if a number is > 0, then that means that there is at least one SNP in the gude
#For each guide, calculate how many cell lines have a SNP

num.lines.with.snp.in.guide = processed.genotype.matrix %>%
select(-snp) %>%
apply(1, function(x) { 
    num.with.snp = x[x != 0] %>% length()
    return(num.with.snp)
})

In [29]:
num.lines.with.snp.in.guide

[1] 256 229 514 591 545 533 181  13 188  17  67  14 398   8  80 356  30  24
  [19] 336  38  89  81 544  22  63 627  22 393  14 555  18 631 438 495 594 303
  [37] 603  93  56  32 284  82  26  81  83 579 190  19  30  87 472  73  35 173
  [55]  18   7  10  39  58  32  12 185 106 167 370 190  12 132 102  31  45 622
  [73] 175  66  15 520  29 113 100  61 614 393 430 297 458 435 512   8 374 320
  [91]  81  48  37  26 165  26 122 544 452  71 590  10 123  71   8 574  59  13
 [109] 141 152 600 358  27 299  15 580  12  10 110 411 484  41   2  14  19 310
 [127] 492 632  42 574  28 509 344 541  37 546  45 189  38 332 366 172  20 161
 [145] 195 384 325 198 432  92  35  66   9 253  29 149  22 208  11   8  18 116
 [163] 317 349  29  14  20  13 144  26 368 538   5  12 139 126 503  23  17 257
 [181] 633  15  93   9 219  87 231  38 562 148  11 320 213   5  53  22  27 524
 [199]  21  67  38  41  37  43 177 289  61  13  39  13 277 175 498 185 134  83
 [217]   6 424 172  46 301 361  71 296 411  13  42  15 165  12 612  15 186  11
 [235]  24  26  43 595  60 489  11 416 538  24  77 402   7  18   9 286 396  23
 [253]  92 633  48 271  16  19 223 253 424  14  27 105 323   8 365 115 362 450
 [271]  17 105 184 331  89 367  23 114  13 326 103  12 525  15 110  84 248   9
 [289] 594  17 124  11 255  28 633 560 295 540 126  14  20 271  90 282 340  60
 [307] 147 342 573 240 622 211 344   9 164 303   5 118   7  10  25  24 411 208
 [325] 562 275 622 485 192  25 589 219  11 404 205  15  15 633 137 336  64 524
 [343]  13 145 626  43 102  20  20 204 483  39  48  21  17  63 399  19  80 632
 [361] 163 309  17 105  21 633 253  49 148 629 129 188 157 298 246  48  11 143
 [379]  80 136  22 262 254 104   8  33 244  83 153 304 212 432  39  68 566 565
 [397]  58  15 499  74 424 441 274  42  10 174  33 633 277   9 545  60   7 186
 [415] 108 200 198  13 183  43  21 305  38 613  32 354  54  10  37  54 112  17
 [433] 224 131 379  12 194  10 190  66  19 216  50  16 135  12 391  40  78 138
 [451] 461  12 319 258 565 583  46 131  23 633  82  39  73 485  21  62 284  63
 [469]  14  28  15 335 268  29   9 138  12  48 115  46 332  20  58  22  85 100
 [487] 109  55  15  11  13  28 555  70 358   7 115 573 565 206 202 570   7  24
 [505] 299 175  36 154  18 620 531 486 444  12 512 498  85 380 362  10 545  12
 [523] 589 265  58  14 525  86 108  21  16 227  19  67 363  75  19   9 347   9
 [541]  17  53 386 136 514 631 631 214  68 632 495 493 622 582 314  64 555 533
 [559] 383 178 417  11 442 225 545 174 420  38  79 575  16  19 553  17  76  99
 [577]  64  24 239 206  25  31 181 282  10  16  11 531  19  73   8  77 105 522
 [595] 274  29   1  73  25 324  14 331  53   8 174  20 101  43  31 215 510  50
 [613] 482  14 122 176  10  27  39 562 544  14  13  10 282  20  23 284  15   8
 [631]  33  78 154 514   9  46 168  48  10  68  95  63   7   9  56  36 424  13
 [649]  33  53 147 505 170 483 137   7  15   8   9  53  16  97  29  17  97  16
 [667] 459  17 429  26  11 428  72 580 339  15  23  94 546  15 630  15 471 106
 [685] 459 232 104 633 439  17 333   9 433 218 306 526 590 209  16 475  25 629
 [703] 476  21  18 165  92   9 560 352 434  10 234  52  19  16 115 536 101 621
 [721] 464 175 230 580  18 189 629 353 267  25 361  92  26 201  11 443  16  64
 [739] 530  40  34 167 258 201  42 466 438  71  17 574 366 278 270 308 289  11
 [757]  15  23 632 443   9  21  38  22 217 267   9 626 189  69  15  41  16 136
 [775] 522 282  27  16 283  18  16  19   3 203 208 119  60 277  81   8 467   6
 [793] 281  15 194 629  10  87  14 215   5  17  15  15 237 152  13  63  11  16
 [811] 345  83  58 443  26 148 393 213  18  45  14 222  25 197 218 470 487 116
 [829] 202  31  40 263 268 622 631 105  60 235  20 455  11 147 633 303  21 114
 [847]  25 172  14  35  11 152 223 449  10  36 318  87 175 180 303  28   4 575
 [865] 379 232 106 379 101  12  15  20 328 554 248 197  59 179  10  34 534  17
 [883]  30 376 486 589 376  50 295 523 280 437  96  74  50  33  98  67  54  21
 [901]  29 604 424  11 535  21  81 108 102  46  15 22

In [30]:
#Calculate the number of guides in depmap
guides.in.depmap = nrow(avana.bed)
guides.in.depmap

#Calculate the fraction of guides with at least one SNP
print("This is the number of guides with a SNP in at least ten lines")
num.with.10.lines = num.lines.with.snp.in.guide[num.lines.with.snp.in.guide >= 10] %>% length()
num.with.10.lines/guides.in.depmap


[1] 72104

[1] "This is the number of guides with a SNP in at least ten lines"


[1] 0.0432847

In [40]:
#Calculate the number of genes that are affected in at least 10 cell lines
affected.in.10.guides = processed.genotype.matrix$snp[num.lines.with.snp.in.guide >= 10]
head(affected.in.10.guides)

#Convert the guides to genes
setwd("/home/jupyter/Ancestry/gwas")
affected.genes =  read.table("internal-21q4_v9-achilles-guide-map.csv", header = T, sep = ",") %>%
select(sgrna, gene) %>%
mutate(gene = word(gene, 1)) %>%
filter(sgrna %in% affected.in.10.guides) %>%
pull(gene) %>%
unique()

head(affected.genes) 
length(affected.genes)

[1] "AAAACTGTACGAGTGACGGA" "AAAGCGGTCACCTTCCACAT" "AAAGGAGTCGAAGAAGTCCG"
[4] "AAAGGATGAAAGACTCCGAA" "AAAGTGCAAGAGGATACCCA" "AAAGTTTCTGTCTTACCTGG"

[1] "VIPR2"    "CLDN6"    "TRAPPC12" "ALPK2"    "SIGLEC12" "DEFB131A"

[1] 2779

In [39]:
setwd("/home/jupyter/Ancestry/gwas")
read.table("internal-21q4_v9-achilles-guide-map.csv", header = T, sep = ",") %>%
select(sgrna, gene) %>%
mutate(gene = word(gene, 1))
head()



sgrna,gene
<chr>,<chr>
AAAAAAATCCAGCAATGCAG,SHOC2
AAAAAACCCGTAGATAGCCT,NDUFA12
AAAAAAGAAGAAAAAACCAG,SDAD1
AAAAAAGCTCAAGAAGGAGG,FAM98A
AAAAAAGGCTGTAAAAGCGT,ZNF253
AAAAAAGGGCTCCAAAAAGG,H2BC7
AAAAACAACACATCAGAGCG,SYNE2
AAAAACTCTGGGAAATGACT,SPHAR
AAAAAGACAACCTCGCCCTG,BATF2


ERROR: Error in checkHT(n, dx <- dim(x)): argument "x" is missing, with no default
